# Peptide Analysis

The following notebook shows the data and code used to plot positively identified peptides in samples provided by Hayley B on 1/21/21.

In [1]:
from commons.data_processing import *
from commons.my_mzml import *
from commons.peaks import *
import os 
import re
import ntpath
import numpy as np 
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt 

In [152]:
# pull the xml data for use
xmls = get_files(directory=r'.\Peptide Identification', exts=['.mzXML'])

# read the data file into dataframe
data = r'.\Peptide Identification\75_25 de novo peptides\de novo peptides.csv'
df = pd.read_csv(data)
df.head()

,Fraction,Source File,Feature,Peptide,Scan,Tag Length,Denovo Score,ALC (%),length,m/z,z,RT,Predict RT,Intensity,Mass,ppm,PTM,local confidence (%),tag (>=0%),mode
0,29,20210124_GD_ZC_HS_MetBGE_500V_75_25_200mz.raw,F29:609,AGGA,F29:1056,4,99,99,4,275.1336,1,3.56,-,1153900.0,274.1277,-4.8,NaN,100 100 100 100,AGGA,HCD
1,30,20210124_GD_ZC_HS_MetBGE_500V_75_25_300mz.raw,F30:185,AGGGG,F30:1084,5,99,99,5,318.1416,1,3.71,-,1227100.0,317.1335,2.7,NaN,100 100 100 100 100,AGGGG,HCD
2,30,20210124_GD_ZC_HS_MetBGE_500V_75_25_300mz.raw,F30:607,NGGGG,F30:1108,5,99,99,5,361.1465,1,3.83,-,637610.0,360.1393,-0.4,NaN,99 100 100 100 100,NGGGG,HCD
3,29,20210124_GD_ZC_HS_MetBGE_500V_75_25_200mz.raw,F29:393,GGGG,F29:966,4,99,99,4,247.1041,1,3.40,-,4681300.0,246.0964,1.6,NaN,99 100 100 100,GGGG,HCD
4,30,20210124_GD_ZC_HS_MetBGE_500V_75_25_300mz.raw,F30:789,NGGGA,F30:1127,5,99,99,5,375.1674,1,3.87,-,330630.0,374.1550,13.7,NaN,99 100 100 100 100,NGGGA,HCD


In [156]:
df.Scan.str.split(':', expand=True)[1]

0      1056
1      1084
2      1108
3       966
4      1127
       ... 
164     864
165    1097
166    1255
167    1106
168    1146
Name: 1, Length: 169, dtype: object

In [157]:
def filter_peps(s):
    '''
    Function to parse peptide sequences and return boolean of
    entities that match our expected output.


    :param s: (str) peptide sequence string
    '''
    s = {let : s.count(let) for let in s.upper()}
    keys = list(s.keys())
    if len(keys) > 2:
        return False
    elif len(keys) == 2:
        if sorted(keys) == ['A', 'G']:
            return True
    elif len(keys) == 1:
        if keys[0] in ['A', 'G']:
            return True
    return False

# check to see if peptide is valid for our experiment
df.loc[:, 'passes'] = df.Peptide.map(filter_peps)

# use pandas string methods to split scan and feature number
# keep only the scan number
df.loc[:, 'Scan'] = df.Scan.str.split(':', expand=True)[1]

valid = df[df.passes==True]

In [158]:
valid

,Fraction,Source File,Feature,Peptide,Scan,Tag Length,Denovo Score,ALC (%),length,m/z,...,RT,Predict RT,Intensity,Mass,ppm,PTM,local confidence (%),tag (>=0%),mode,passes
0,29,20210124_GD_ZC_HS_MetBGE_500V_75_25_200mz.raw,F29:609,AGGA,1056,4,99,99,4,275.1336,...,3.56,-,1153900.0,274.1277,-4.8,NaN,100 100 100 100,AGGA,HCD,True
1,30,20210124_GD_ZC_HS_MetBGE_500V_75_25_300mz.raw,F30:185,AGGGG,1084,5,99,99,5,318.1416,...,3.71,-,1227100.0,317.1335,2.7,NaN,100 100 100 100 100,AGGGG,HCD,True
3,29,20210124_GD_ZC_HS_MetBGE_500V_75_25_200mz.raw,F29:393,GGGG,966,4,99,99,4,247.1041,...,3.40,-,4681300.0,246.0964,1.6,NaN,99 100 100 100,GGGG,HCD,True
5,30,20210124_GD_ZC_HS_MetBGE_500V_75_25_300mz.raw,F30:972,AGGGGA,1135,6,99,99,6,389.1786,...,3.91,-,100540.0,388.1706,1.7,NaN,99 99 100 100 100 99,AGGGGA,HCD,True
6,30,20210124_GD_ZC_HS_MetBGE_500V_75_25_300mz.raw,F30:787,AGGGGG,1127,6,99,99,6,375.1611,...,3.87,-,391300.0,374.1550,-3.0,NaN,98 99 100 100 100 100,AGGGGG,HCD,True
8,30,20210124_GD_ZC_HS_MetBGE_500V_75_25_300mz.raw,-,AAGGA,1110,5,98,98,5,346.1712,...,3.82,-,0.0,345.1648,-2.5,NaN,95 97 100 99 98,AAGGA,HCD,True
10,30,20210124_GD_ZC_HS_MetBGE_500V_75_25_300mz.raw,F30:971,AAGGGG,1135,6,96,96,6,389.1720,...,3.91,-,76629.0,388.1706,-15.2,NaN,95 88 95 100 100 100,AAGGGG,HCD,True
19,29,20210124_GD_ZC_HS_MetBGE_500V_75_25_200mz.raw,F29:610,GGAA,1056,4,91,91,4,275.1375,...,3.56,-,1230300.0,274.1277,9.4,NaN,99 73 94 98,GGAA,HCD,True
33,29,20210124_GD_ZC_HS_MetBGE_500V_75_25_200mz.raw,F29:392,GGGG,966,4,81,81,4,247.1008,...,3.40,-,3719000.0,246.0964,-11.9,NaN,80 74 86 81,GGGG,HCD,True
34,29,20210124_GD_ZC_HS_MetBGE_500V_75_25_200mz.raw,F29:394,GGGG,966,4,81,81,4,247.1074,...,3.40,-,3565800.0,246.0964,15.1,NaN,80 74 86 81,GGGG,HCD,True


In [159]:
# this function is typically called from my 'commons' library because I use it often
# I have displayed it here for illustration

def plot_ms2_data(xs, ys, peptide, frag_dict, mods=None, show_error=False, tolerance=25):
    '''
    Function to return altair plot of identified fragments for a theoretical 
    peptide.

    :param xs: (array) x/time data
    :param ys: (array) intensity data
    :param peptide: (string) peptide sequence
    :param frag_dict: (dict) output returned from data_processing.fragments func
    '''
    df = pd.DataFrame({
        'x':xs, 'y':ys,
        'fragment':['None']*len(xs),
        'label':['']*len(xs)
    })
    df.loc[:, 'y'] = df.y / np.max(df.y) * 100
    df['label position'] = df.y + 5

    err_mass, err_dist, err_kind= [], [], []

    if mods is not None:
        assert isinstance(mods, dict), 'modifications must enter as dictionary'
        for k in mods:
            frag_dict[k] = mods[k]

    for k, v in frag_dict.items():
        for frag in v:
            nearest = find_nearest(df.x, frag)
            error = mass_error(frag, nearest)
            if abs(error) <= tolerance:
                err_mass.append(nearest)
                err_dist.append(error)
                err_kind.append(k)
                df.loc[(df.x==nearest), 'fragment'] = k
                df.loc[(df.x==nearest), 'label'] = k+f'{v.index(frag)+1}'
    
    dom = {
        'b': '#3b5bad',
        'y': '#c42e23'
    }
    
    bars = alt.Chart(df).mark_bar(size=2).encode(
        x=alt.X('x', title='m/z', axis=alt.Axis(grid=False)),
        y=alt.Y('y', title='Relative Abundance',
                axis=alt.Axis(grid=False, tickCount=1),
                scale=alt.Scale(domain=(0, 100))),
        color=alt.Color('fragment', scale=alt.Scale(domain=list(dom.keys()),
                        range=list(dom.values())), legend=None)
    ).properties(
        title=peptide
    )

    text = alt.Chart(df).mark_text().encode(
        y=alt.Y('label position'),
        x=alt.X('x'),
        text='label'
    )
    chart = alt.vconcat()
    chart &= alt.layer(bars, text)
    
    if show_error:
        err_df = pd.DataFrame({
            'mass':err_mass,
            'error':err_dist,
            'kind':err_kind
        })
        dots = alt.Chart(err_df).mark_circle().encode(
            x=alt.X('mass:Q', title='m/z', axis=alt.Axis(grid=False)),
            y=alt.Y('error:Q', title='error (ppm)', axis=alt.Axis(grid=True, tickCount=3),
                    scale=alt.Scale(domain=(-tolerance, tolerance))),
            color=alt.Color('kind:O', scale=alt.Scale(domain=list(dom.keys()), range=list(dom.values()))
            )).properties(height=100)

        line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(
            strokeDash=[10, 10]).encode(y='y')


        chart &= (dots + line)
    
    return chart

In [161]:
chart = alt.vconcat()

for item in zip(valid['Source File'], valid['Peptide'],
                valid['Scan']):
    source = item[0]
    # print(source)
    source = source.split('.')[0]
    source = [f for f in xmls if re.search(r'{}'.format(source), f)][0]
    m = mzXML(r'{}'.format(source))


    peptide = item[1]
    frags = fragments(peptide)

    scan = item[2]
    x, y = m.get_scan(scan)
    x, y = prof_to_cent(x, y)

    chart &= plot_ms2_data(x, y, peptide, frags, show_error=True).resolve_scale(
        x='shared'
    )
    
chart.configure_view(
        strokeWidth=0
    )

alt.VConcatChart(...)